# **Setup Library**

In [ ]:
# Requirements, will skip if already installed
%pip install geopandas rasterio rasterstats shapely
%pip install folium earthengine-api
%pip install scikit-learn
%pip install treeinterpreter

In [ ]:
from os import path as op
import pickle

import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.features import rasterize
from rasterstats.io import bounds_window
import rasterstats
import folium

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from treeinterpreter import treeinterpreter as ti

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# your root directory for outputs is set to your google drive
# you should create a sub-folder called data under your 'Colab Notebooks'.
# An example is in https://developmentseed.org/sat-ml-training/GettingStarted#Explore-your-drive
my_root_dir = #"/content/drive/MyDrive/Colab Notebooks/Data/"

# **Reprojection**

In [ ]:
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas

src_file = #'/content/drive/MyDrive/Colab Notebooks/Data/S2.tif'
dst_file = #'/content/drive/MyDrive/Colab Notebooks/Data/S2_4326.tif'
dst_crs = {'init': 'EPSG:4326'}

with rasterio.open(src_file) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(dst_file, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)



---



# **Prepare Data & Training Data**

In [ ]:
##If you want to read the data directly from the shared folder, uncomment the following line.
raster_file = '/content/drive/MyDrive/Colab Notebooks/Data/S2_4326.tif'

full_dataset = rasterio.open(raster_file)
img_rows, img_cols = full_dataset.shape
img_bands = full_dataset.count
print(full_dataset.shape) # dimensions
print(full_dataset.count) # bands
print(full_dataset.crs)



---



# **Prepare Vector**

In [ ]:
# read in training data polygons that created as geojson from a shared directory
training_data = '/content/drive/MyDrive/Colab Notebooks/Data/Training_data_4326.geojson'
training_vectors = gpd.read_file(training_data)

# make a bounding box and centroid for mapping
bbox = training_vectors.total_bounds
center = shp.geometry.box(bbox[0], bbox[1], bbox[2], bbox[3]).centroid

# show the 1st 5 lines
training_vectors.head()

len(training_vectors)

points_clean = training_vectors[training_vectors.geometry.type == 'Point']

len(training_vectors)

# find all unique values of training data names to use as classes
classes = np.unique(points_clean.NAME)
print(classes)

# create a dictionary to convert class names into integers for modeling
class_dict = dict(zip(classes, range(len(classes))))
print(class_dict)    

print(len(training_vectors))



---



In [ ]:
# raster information
# a custom function for getting each value from the raster
def all_values(x):
    return x

# this larger cell reads data from a raster file for each training vector
X_raw = []
y_raw = []
with rasterio.open(raster_file, 'r') as src:
    for (label, geom) in zip(points_clean.NAME, points_clean.geometry):
        
        # read the raster data matching the geometry bounds
        window = bounds_window(geom.bounds, src.transform)
        # store our window information
        window_affine = src.window_transform(window)
        fsrc = src.read(window=window)
        # rasterize the geometry into the larger shape and affine
        mask = rasterize(
            [(geom, 1)],
            out_shape=fsrc.shape[1:],
            transform=window_affine,
            fill=0,
            dtype='float',
            all_touched=True
        ).astype(bool)
        
        # for each label pixel (places where the mask is true)
        label_pixels = np.argwhere(mask)
        
        for (row, col) in label_pixels:
            # add a pixel of data to X
            data = fsrc[:,row,col]
            one_x = np.nan_to_num(data, nan=1e-3)
            X_raw.append(one_x)
            # add the label to y
            y_raw.append(class_dict[label])

In [ ]:
# convert the training data lists into the appropriate numpy array shape and format for scikit-learn
X = np.array(X_raw)
y = np.array(y_raw)
print(X.shape, y.shape)
print(type(X), type(y))

In [ ]:
#from sklearn import preprocessing
#min_max_scaler = preprocessing.MinMaxScaler()
#X = min_max_scaler.fit_transform(X)
#y = np.ravel(y)
#y.reshape(-1)



---



---



In [ ]:
# split the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
# calculate class weights to allow for training on inbalanced training samples
labels, counts = np.unique(y_train, return_counts=True)
class_weight_dict = dict(zip(labels, 1 / counts))
class_weight_dict

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)



---



# **Machine Learning**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
########################################################
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
########################################################
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster, reshape_as_image
import sklearn

In [ ]:
SVC_model = SVC()
KNN_model = KNeighborsClassifier()
GNB_model = GaussianNB()
DT_model = DecisionTreeClassifier()
RF_model = RandomForestClassifier()
GB_model = GradientBoostingClassifier()
MLP_model = MLPClassifier()
XGB_model = XGBClassifier()
deep_MLP_model = MLPClassifier(hidden_layer_sizes=(int(full_dataset.count),200,200,200,labels.size),activation="relu" ,random_state=1, max_iter=1000)

In [ ]:
SVC_model.fit(X_train, y_train)
KNN_model.fit(X_train, y_train)
GNB_model.fit(X_train, y_train)
DT_model.fit(X_train, y_train)
RF_model.fit(X_train, y_train)
GB_model.fit(X_train, y_train)
MLP_model.fit(X_train, y_train)
XGB_model.fit(X_train, y_train)

In [ ]:
SVC_prediction = SVC_model.predict(X_test)
KNN_prediction = KNN_model.predict(X_test)
GNB_prediction = GNB_model.predict(X_test)
DT_prediction = DT_model.predict(X_test)
RF_prediction = RF_model.predict(X_test)
GB_prediction = GB_model.predict(X_test)
MLP_prediction = MLP_model.predict(X_test)
XGB_prediction = XGB_model.predict(X_test)



---




# **Performance**

In [ ]:
SVC_accuracy = accuracy_score(SVC_prediction, y_test)
print("Accuracy: %.2f%%" % (SVC_accuracy * 100.0))
print(classification_report(SVC_prediction, y_test))
print(confusion_matrix(SVC_prediction, y_test))
#report_SVC = pd.DataFrame(classification_report(SVC_prediction, y_test, digits=2,output_dict=True)).T
#report_SVC['support'] = report_SVC.support.apply(int)
#report_SVC.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "SVC_"  + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(SVC_prediction, y_test),display_labels=SVC_model.classes_)
disp.plot()
plt.show()

In [ ]:
KNN_accuracy = accuracy_score(KNN_prediction, y_test)
print("Accuracy: %.2f%%" % (KNN_accuracy * 100.0))
print(classification_report(KNN_prediction, y_test))
print(confusion_matrix(KNN_prediction, y_test))

#report_KNN = pd.DataFrame(classification_report(KNN_prediction, y_test, digits=2,output_dict=True)).T
#report_KNN['support'] = report_KNN.support.apply(int)
#report_KNN.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "KNN_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(KNN_prediction, y_test),display_labels=KNN_model.classes_)
disp.plot()
plt.show()

In [ ]:
GNB_accuracy = accuracy_score(GNB_prediction, y_test)
print("Accuracy: %.2f%%" % (GNB_accuracy * 100.0))
print(classification_report(GNB_prediction, y_test))
print(confusion_matrix(GNB_prediction, y_test))

#report_GNB = pd.DataFrame(classification_report(GNB_prediction, y_test, digits=2,output_dict=True)).T
#report_GNB['support'] = report_GNB.support.apply(int)
#report_GNB.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "GNB_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(GNB_prediction, y_test),display_labels=GNB_model.classes_)
disp.plot()
plt.show()

In [ ]:
DT_accuracy = accuracy_score(DT_prediction, y_test)
print("Accuracy: %.2f%%" % (DT_accuracy * 100.0))
print(classification_report(DT_prediction, y_test))
print(confusion_matrix(DT_prediction, y_test))

#report_DT = pd.DataFrame(classification_report(DT_prediction, y_test, digits=2,output_dict=True)).T
#report_DT['support'] = report_DT.support.apply(int)
#report_DT.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "DT_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(DT_prediction, y_test),display_labels=DT_model.classes_)
disp.plot()
plt.show()

In [ ]:
RF_accuracy = accuracy_score(RF_prediction, y_test)
print("Accuracy: %.2f%%" % (RF_accuracy * 100.0))
print(classification_report(RF_prediction, y_test))
print(confusion_matrix(RF_prediction, y_test))

#report_RF = pd.DataFrame(classification_report(RF_prediction, y_test, digits=2,output_dict=True)).T
#report_RF['support'] = report_RF.support.apply(int)
#report_RF.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "RF_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(RF_prediction, y_test),display_labels=RF_model.classes_)
disp.plot()
plt.show()

In [ ]:
GB_accuracy = accuracy_score(GB_prediction, y_test)
print("Accuracy: %.2f%%" % (GB_accuracy * 100.0))
print(classification_report(GB_prediction, y_test))
print(confusion_matrix(GB_prediction, y_test))

#report_GB = pd.DataFrame(classification_report(GB_prediction, y_test, digits=2,output_dict=True)).T
#report_GB['support'] = report_GB.support.apply(int)
#report_GB.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "GB_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(GB_prediction, y_test),display_labels=GB_model.classes_)
disp.plot()
plt.show()

In [ ]:
MLP_accuracy = accuracy_score(MLP_prediction, y_test)
print("Accuracy: %.2f%%" % (MLP_accuracy * 100.0))
print(classification_report(MLP_prediction, y_test))
print(confusion_matrix(KNN_prediction, y_test))

#report_MLP = pd.DataFrame(classification_report(MLP_prediction, y_test, digits=2,output_dict=True)).T
#report_MLP['support'] = report_MLP.support.apply(int)
#report_MLP.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "MLP_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(MLP_prediction, y_test),display_labels=MLP_model.classes_)
disp.plot()
plt.show()

In [ ]:
XGB_accuracy = accuracy_score(XGB_prediction, y_test)
print("Accuracy: %.2f%%" % (XGB_accuracy * 100.0))
print(classification_report(XGB_prediction, y_test))
print(confusion_matrix(KNN_prediction, y_test))

#report_XGB = pd.DataFrame(classification_report(XGB_prediction, y_test, digits=2,output_dict=True)).T
#report_XGB['support'] = report_XGB.support.apply(int)
#report_XGB.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Data/classification_report/' + exp_no + "XGB_" + polarization +  '.csv',encoding='utf-8-sig')

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(XGB_prediction, y_test),display_labels=XGB_model.classes_)
disp.plot()
plt.show()



---



# **Tuning Hyper Parameter**

In [ ]:
# import the necessary packages
from sklearn.model_selection import GridSearchCV

In [ ]:
SVC_param_grid = {'C': [0.1, 1, 10, 100, 1000], #5
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], #5
              'kernel': ['rbf']}
#Grid Search
SVC_grid_cv = GridSearchCV(SVC(), SVC_param_grid, cv=10, scoring="accuracy", n_jobs=-1)
SVC_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(SVC_grid_cv.best_params_)

#Test Accuracy
SVC_grid_cv_prediction = SVC_grid_cv.predict(X_test)
SVC_grid_cv_accuracy = accuracy_score(SVC_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (SVC_grid_cv_accuracy * 100.0))
print(classification_report(SVC_grid_cv_prediction, y_test))
print(confusion_matrix(SVC_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_SVC_grid_cv_13042022.sav'
pickle.dump(SVC_grid_cv, open(filename, 'wb'))

In [ ]:
KNN_param_grid = {'leaf_size': np.arange(1, 50, step=10), #5
              'n_neighbors': np.arange(1, 50, step=10), #5
              'p': [1,2,3,4,5]}

#Grid Search
KNN_grid_cv = GridSearchCV(KNeighborsClassifier(), KNN_param_grid, cv=10, scoring="accuracy", n_jobs=-1)
KNN_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(KNN_grid_cv.best_params_)

#Test Accuracy
KNN_grid_cv_prediction = KNN_grid_cv.predict(X_test)
KNN_grid_cv_accuracy = accuracy_score(KNN_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (KNN_grid_cv_accuracy * 100.0))
print(classification_report(KNN_grid_cv_prediction, y_test))
print(confusion_matrix(KNN_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_KNN_grid_cv_13042022.sav'
pickle.dump(KNN_grid_cv, open(filename, 'wb'))

In [ ]:
RF_param_grid = {
    "n_estimators": [5,20,50,100], #np.arange(1, 400, step=50),
    'max_features': ['auto', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(10, 120, num = 12)],
    'min_samples_split': [2, 6, 10],
    'min_samples_leaf': [1, 3, 4],
    'bootstrap': [True, False],
    "criterion" : ["gini", "entropy"],
}

#Grid Search
RF_grid_cv = GridSearchCV(RandomForestClassifier(), RF_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
RF_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(RF_grid_cv.best_params_)

RF_grid_cv_prediction = RF_grid_cv.predict(X_test)
RF_grid_cv_accuracy = accuracy_score(RF_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (RF_grid_cv_accuracy * 100.0))
print(classification_report(RF_grid_cv_prediction, y_test))
print(confusion_matrix(RF_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_RF_grid_cv_13042022.sav'
pickle.dump(RF_grid_cv, open(filename, 'wb'))

In [ ]:
GB_param_grid = {
      'learning_rate':[0.1,0.01],
      'n_estimators':[100,500,1000],
      'max_depth':[10,50,100],
      #'min_samples_split':[2,4,6,8,10],
      #'min_samples_leaf':[1,3,5,7,9],
      #'max_features':[2,3,4,5,6],
      #'subsample':[0.5,0.6,0.7,0.8,0.9]
}

#Grid Search
GB_grid_cv = GridSearchCV(GradientBoostingClassifier(), GB_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
GB_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(GB_grid_cv.best_params_)

GB_grid_cv_prediction = GB_grid_cv.predict(X_test)
GB_grid_cv_accuracy = accuracy_score(GB_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (GB_grid_cv_accuracy * 100.0))
print(classification_report(GB_grid_cv_prediction, y_test))
print(confusion_matrix(GB_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_GB_grid_cv_13042022.sav'
pickle.dump(GB_grid_cv, open(filename, 'wb'))

In [ ]:
#Tuning Example (3 parameter)
MLP_param_grid = {
    "hidden_layer_sizes": np.arange(100, 1000, step=100),
    "activation": ['relu'],
    'solver': ['adam'],
    'learning_rate': ['constant','adaptive'],
}

#Grid Search
MLP_grid_cv = GridSearchCV(MLPClassifier(), MLP_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
MLP_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(MLP_grid_cv.best_params_)

MLP_grid_cv_prediction = MLP_grid_cv.predict(X_test)
MLP_grid_cv_accuracy = accuracy_score(MLP_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (MLP_grid_cv_accuracy * 100.0))
print(classification_report(MLP_grid_cv_prediction, y_test))
print(confusion_matrix(MLP_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_MLP_grid_cv_13042022.sav'
pickle.dump(MLP_grid_cv, open(filename, 'wb'))

In [ ]:
XGB_param_grid = {
      'n_estimators':np.arange(100, 1000, step=100),
      'max_depth':np.arange(10, 100, step=10),
      'learning_rate':[0.01],
      #'min_child_weight' : [1, 3, 5],
      #'subsample': [0.7,0.8,0.9],
      #'colsample_bytree': [0.5, 0.7],
}

#Grid Search
XGB_grid_cv = GridSearchCV(XGBClassifier(), XGB_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
XGB_grid_cv.fit(X_train, y_train)
print("Best params:\n")
print(XGB_grid_cv.best_params_)

XGB_grid_cv_prediction = XGB_grid_cv.predict(X_test)
XGB_grid_cv_accuracy = accuracy_score(XGB_grid_cv_prediction, y_test)
print("Accuracy: %.2f%%" % (XGB_grid_cv_accuracy * 100.0))
print(classification_report(XGB_grid_cv_prediction, y_test))
print(confusion_matrix(XGB_grid_cv_prediction, y_test))

In [ ]:
# save the model to disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_XGB_grid_cv_13042022.sav'
pickle.dump(XGB_grid_cv, open(filename, 'wb'))



---



# **Deep Learning**

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
########################################################
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l2
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn import metrics
import sklearn
from sklearn.metrics import ConfusionMatrixDisplay
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# **FCN**

In [ ]:
model_FCN = tf.keras.Sequential([
    #tf.keras.layers.Dense(1024, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    #tf.keras.layers.Dropout(0.1),      
    tf.keras.layers.Dense(512, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),           
    tf.keras.layers.Dense(256, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1), 
    tf.keras.layers.Dense(128, activation='relu',activity_regularizer=regularizers.l2(0.01)), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(16, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8, activation='relu',activity_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(5, activation='softmax'),         
])
#,activity_regularizer=regularizers.l2(0.01)
#tf.keras.layers.Dropout(0.2),   

In [ ]:
model_FCN.build(X_train.shape)

In [ ]:
print(model_FCN.summary())

In [ ]:
model_FCN.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
# Fit model  
epochs = 1000           
history_FCN = model_FCN.fit(X_train, y_train,epochs=epochs, batch_size=128,validation_split = 0.2, shuffle=True)

In [ ]:
# list all data in history
print(history_FCN.history.keys())

In [ ]:
acc = history_FCN.history['accuracy']
val_acc = history_FCN.history['val_accuracy']

loss = history_FCN.history['loss']
val_loss = history_FCN.history['val_loss']

In [ ]:
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history_FCN.history['loss'])
plt.plot(history_FCN.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# Model evaluation with test data set 
# Prediction at test data set
Y_pred = model_FCN.predict(X_test)
Y_predict_class = tf.argmax(Y_pred, axis=-1)
print(Y_predict_class)
score = model_FCN.evaluate(X_test, y_test,batch_size=10, verbose=1)
print(score)
print("Baseline Error: %.2f%%" % (100-score[1]*100))

In [ ]:
FCN_accuracy = accuracy_score(Y_predict_class, y_test)
print("Accuracy: %.2f%%" % (FCN_accuracy * 100.0))
print(classification_report(Y_predict_class, y_test))
print(confusion_matrix(Y_predict_class, y_test))

In [ ]:
#model_FCN.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_FCN_17032022')



---



# **RNN**

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
nrows, ncols = X_train.shape
nrows1, ncols1 = X_test.shape
X_train_re = X_train.reshape(nrows, ncols, 1)
X_test_re = X_test.reshape(nrows1, ncols1, 1)
print(X_train_re.shape)
print(X_test_re.shape)

In [ ]:
model_RNN = tf.keras.Sequential()
model_RNN.add(tf.keras.layers.SimpleRNN(56,return_sequences= True)),
model_RNN.add(tf.keras.layers.SimpleRNN(56)),
model_RNN.add(tf.keras.layers.Dense(5, activation='softmax'))

#,activity_regularizer=regularizers.l2(0.1)
#model_RNN.add(tf.keras.layers.Dropout(0.2)), 

In [ ]:
model_RNN.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model_RNN.build(X_train_re.shape)

In [ ]:
print(model_RNN.summary())

In [ ]:
# Fit model  
epochs = 1000            
history_RNN = model_RNN.fit(X_train_re, y_train,epochs=epochs, batch_size=128,validation_split = 0.2, shuffle=True)

In [ ]:
# list all data in history
print(history_RNN.history.keys())

In [ ]:
acc = history_RNN.history['accuracy']
val_acc = history_RNN.history['val_accuracy']

loss = history_RNN.history['loss']
val_loss = history_RNN.history['val_loss']

In [ ]:
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history_RNN.history['loss'])
plt.plot(history_RNN.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# Model evaluation with test data set 
# Prediction at test data set
Y_pred = model_RNN.predict(X_test_re)
Y_predict_class = tf.argmax(Y_pred, axis=-1)
score = model_RNN.evaluate(X_test_re, y_test,batch_size=10, verbose=1)
print(score)
print("Baseline Error: %.2f%%" % (100-score[1]*100))

In [ ]:
RNN_accuracy = accuracy_score(Y_predict_class, y_test)
print("Accuracy: %.2f%%" % (RNN_accuracy * 100.0))
print(classification_report(Y_predict_class, y_test))
print(confusion_matrix(Y_predict_class, y_test))
print(sklearn.metrics.cohen_kappa_score(Y_predict_class, y_test))

labels = ["0", "1", "2", "3", "4"]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(Y_predict_class, y_test),display_labels=labels)
disp.plot()
plt.show()

In [ ]:
#model_RNN.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_RNN_Jan_Jul_2019_28042022')
#model_RNN.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_RNN_MinMaxScaler_22032022')


---



# **LSTM**

In [ ]:
model_LSTM = tf.keras.Sequential()
model_LSTM.add(tf.keras.layers.LSTM(56,return_sequences= True)),
model_LSTM.add(tf.keras.layers.LSTM(56)),
model_LSTM.add(tf.keras.layers.Dense(5, activation='softmax'))
#,dropout=0.1
#,activity_regularizer=regularizers.l2(0.1)

In [ ]:
model_LSTM.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model_LSTM.build(X_train_re.shape)

In [ ]:
print(model_LSTM.summary())

In [ ]:
# Fit model  
epochs = 1000            
history_LSTM = model_LSTM.fit(X_train_re, y_train,epochs=epochs, batch_size=128,validation_split = 0.4, shuffle=True)

In [ ]:
# list all data in history
print(history_LSTM.history.keys())

In [ ]:
acc = history_LSTM.history['accuracy']
val_acc = history_LSTM.history['val_accuracy']

loss = history_LSTM.history['loss']
val_loss = history_LSTM.history['val_loss']

In [ ]:
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history_LSTM.history['loss'])
plt.plot(history_LSTM.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# Model evaluation with test data set 
# Prediction at test data set
Y_pred = model_LSTM.predict(X_test_re)
Y_predict_class = tf.argmax(Y_pred, axis=-1)
score = model_LSTM.evaluate(X_test_re, y_test,batch_size=10, verbose=1)
print(score)
print("Baseline Error: %.2f%%" % (100-score[1]*100))

In [ ]:
lstm_accuracy = accuracy_score(Y_predict_class, y_test)
print("Accuracy: %.2f%%" % (lstm_accuracy * 100.0))
print(classification_report(Y_predict_class, y_test))
print(confusion_matrix(Y_predict_class, y_test))

print(sklearn.metrics.cohen_kappa_score(Y_predict_class, y_test))

labels = ["0", "1", "2", "3", "4"]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(Y_predict_class, y_test),display_labels=labels)
disp.plot()
plt.show()

In [ ]:
model_LSTM.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_LSTM_Jan_Jul_2019_28042022')
#model_LSTM.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_LSTM_31032022_tune')
#model_LSTM.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_LSTMMinMaxScaler_22032022')

---------------------

# Bidirectional LSTM

In [ ]:
model_BI_LSTM = tf.keras.Sequential()
model_BI_LSTM.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(56,return_sequences= True))),
model_BI_LSTM.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(56))),
model_BI_LSTM.add(tf.keras.layers.Dense(5, activation='softmax'))
#,dropout=0.1
#,activity_regularizer=regularizers.l2(0.1)

In [ ]:
model_BI_LSTM.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [ ]:
model_BI_LSTM.build(X_train_re.shape)

In [ ]:
print(model_BI_LSTM.summary())

In [ ]:
# Fit model  
epochs = 1000            
history_BI_LSTM = model_BI_LSTM.fit(X_train_re, y_train,epochs=epochs, batch_size=128,validation_split = 0.4, shuffle=True)

In [ ]:
# list all data in history
print(history_BI_LSTM.history.keys())

In [ ]:
acc = history_BI_LSTM.history['accuracy']
val_acc = history_BI_LSTM.history['val_accuracy']

loss = history_BI_LSTM.history['loss']
val_loss = history_BI_LSTM.history['val_loss']

In [ ]:
epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history_BI_LSTM.history['loss'])
plt.plot(history_BI_LSTM.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
# Model evaluation with test data set 
# Prediction at test data set
Y_pred = model_BI_LSTM.predict(X_test_re)
Y_predict_class = tf.argmax(Y_pred, axis=-1)
score = model_BI_LSTM.evaluate(X_test_re, y_test,batch_size=10, verbose=1)
print(score)
print("Baseline Error: %.2f%%" % (100-score[1]*100))

In [ ]:
BI_lstm_accuracy = accuracy_score(Y_predict_class, y_test)
print("Accuracy: %.2f%%" % (BI_lstm_accuracy * 100.0))
print(classification_report(Y_predict_class, y_test))
print(confusion_matrix(Y_predict_class, y_test))

print(sklearn.metrics.cohen_kappa_score(Y_predict_class, y_test))

labels = ["0", "1", "2", "3", "4"]
disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(Y_predict_class, y_test),display_labels=labels)
disp.plot()
plt.show()

In [ ]:
model_BI_LSTM.save('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_BI_LSTM_Jan_Jul_2019_28042022')



---




# Load Model

In [ ]:
# load the model from disk
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/MLP_grid_cv_14022022.sav'

#filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_SVC_grid_cv_13042022.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)



---





# **Prepare image for predict**

In [ ]:
with rasterio.open(raster_file) as src:
    # may need to reduce this image size if your kernel crashes, takes a lot of memory
    img = src.read()[:, :, :]

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
print(img.shape)
reshaped_img = reshape_as_image(img)
print(reshaped_img.shape)



---



# **ML Predict**

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/Model_save/Machine_Learning/Sentinel-1_Jan_Jul_2019_KNN_grid_cv_13042022.sav'
loaded_model_KNN_grid_cv = pickle.load(open(filename, 'rb'))

#class_prediction = RF_model.predict(reshaped_img.reshape(-1, 24))
class_prediction = loaded_model_KNN_grid_cv.predict(reshaped_img.reshape(-1, 14))
# Reshape our classification map back into a 2D matrix so we can visualize it
class_prediction = class_prediction.reshape(reshaped_img[:, :, 0].shape)

with rasterio.open(
    '/content/drive/MyDrive/Colab Notebooks/result/Result.tif',
    'w',
    driver='GTiff',
    height=class_prediction.shape[0],
    width=class_prediction.shape[1],
    count=1,
    dtype='float32'
) as dst:
    dst.write(class_prediction, 1)



---



# **DL Predict**

In [ ]:
with rasterio.open(raster_file) as src:
    # may need to reduce this image size if your kernel crashes, takes a lot of memory
    img = src.read()[:, :, :]

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
print(img.shape)
reshaped_img = reshape_as_image(img)
print(reshaped_img.shape)

# **FCN Predict Image**

In [ ]:
from tensorflow import keras
#model_FCN = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_FCN_17032022')
#class_prediction = model_FCN.predict(reshaped_img.reshape(-1, 24))
class_prediction = model_FCN.predict(reshaped_img.reshape(-1, 8))
class_prediction_argmax = np.argmax(class_prediction, axis=-1)
class_prediction_argmax_reshape = class_prediction_argmax.reshape(reshaped_img[:, :, 0].shape)
with rasterio.open(
    '/content/drive/MyDrive/Colab Notebooks/result/Sentinel-1_Jan_Dec_2019_FCN_model_06042022.tif',
    'w',
    driver='GTiff',
    height=class_prediction_argmax_reshape.shape[0],
    width=class_prediction_argmax_reshape.shape[1],
    count=1,
    dtype='float32'
) as dst:
    dst.write(class_prediction_argmax_reshape, 1)

# **RNN Predict Image**

In [ ]:
from tensorflow import keras
#model_RNN = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_RNN_17032022')
#class_prediction = model_RNN.predict(reshaped_img.reshape(-1, 24))
class_prediction = model_RNN.predict(reshaped_img.reshape(-1,14))
class_prediction_argmax = np.argmax(class_prediction, axis=-1)
class_prediction_argmax_reshape = class_prediction_argmax.reshape(reshaped_img[:, :, 0].shape)
with rasterio.open(
    '/content/drive/MyDrive/Colab Notebooks/result/Sentinel-1_Jan_Jul_2019_RNN_model_28042022.tif',
    'w',
    driver='GTiff',
    height=class_prediction_argmax_reshape.shape[0],
    width=class_prediction_argmax_reshape.shape[1],
    count=1,
    dtype='float32'
) as dst:
    dst.write(class_prediction_argmax_reshape, 1)

# **LSTM Predict image**

In [ ]:
from tensorflow import keras
#model_LSTM = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_LSTM_31032022_tune')
class_prediction = model_LSTM.predict(reshaped_img.reshape(-1, 14))
#class_prediction = model_LSTM.predict(reshaped_img.reshape(-1, 8))
class_prediction_argmax = np.argmax(class_prediction, axis=-1)
class_prediction_argmax_reshape = class_prediction_argmax.reshape(reshaped_img[:, :, 0].shape)
with rasterio.open(
    '/content/drive/MyDrive/Colab Notebooks/result/Sentinel-1_Jan_Jul_2019_LSTM_model_28042022.tif',
    'w',
    driver='GTiff',
    height=class_prediction_argmax_reshape.shape[0],
    width=class_prediction_argmax_reshape.shape[1],
    count=1,
    dtype='float32'
) as dst:
    dst.write(class_prediction_argmax_reshape, 1)

# **BI LSTM Predict image**

In [ ]:
from tensorflow import keras
#model_LSTM = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Model_save/Deep_Learning/model_LSTM_31032022_tune')
class_prediction = model_BI_LSTM.predict(reshaped_img.reshape(-1, 14))
class_prediction_argmax = np.argmax(class_prediction, axis=-1)
class_prediction_argmax_reshape = class_prediction_argmax.reshape(reshaped_img[:, :, 0].shape)
with rasterio.open(
    '/content/drive/MyDrive/Colab Notebooks/result/Sentinel-1_Jan_Jul_2019_BI_LSTM_model_28042022.tif',
    'w',
    driver='GTiff',
    height=class_prediction_argmax_reshape.shape[0],
    width=class_prediction_argmax_reshape.shape[1],
    count=1,
    dtype='float32'
) as dst:
    dst.write(class_prediction_argmax_reshape, 1)